# Kepler orbits

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from sympy import symbols, diff, Matrix
import sympy as sp

E = -0.6

X,Y = symbols('x y')
xvec = [X,Y]
# generate metric from Kepler law
G = 2*(E+1/sp.sqrt(X**2+Y**2))*Matrix(2,2, [1,0,0,1])

Ginv = G.inv()
chr1 = [0.5*(diff(G[k,j],xvec[i])+diff(G[i,k],xvec[j])-diff(G[i,j],xvec[k])) for i in range(2) for j in range(2) for k in range(2)]
chr2 = [sum(Ginv[k,p]*chr1[4*i+2*j+p] for p in range(2)) for i in range(2) for j in range(2) for k in range(2)]


# define right-hand side (s... time, v=[gamma^1,gamma^2,v^1,v^2])
def rhs(s, v): 
    chr111 = chr2[4*0+2*0+0].subs(Y, v[1]).subs(X,v[0])
    chr121 = chr2[4*0+2*1+0].subs(Y, v[1]).subs(X,v[0])
    chr221 = chr2[4*1+2*1+0].subs(Y, v[1]).subs(X,v[0])
    chr112 = chr2[4*0+2*0+1].subs(Y, v[1]).subs(X,v[0])
    chr122 = chr2[4*0+2*1+1].subs(Y, v[1]).subs(X,v[0])
    chr222 = chr2[4*1+2*1+1].subs(Y, v[1]).subs(X,v[0])
    return [v[2],v[3],-v[2]**2*chr111-2*v[2]*v[3]*chr121-v[3]**2*chr221,-v[2]**2*chr112-2*v[2]*v[3]*chr122-v[3]**2*chr222]


Tend = 20
refval = solve_ivp(rhs, (0, Tend), [0.8,0,-0.4,0.5],t_eval=np.linspace(0, Tend,2000), method="DOP853")
plt.plot(refval.y.T[:,0],refval.y.T[:,1])
plt.gca().add_patch(plt.Circle((0, 0), 0.1, color='r'))
plt.axis('equal')
plt.show()

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt

circ1 = Circle((0,0),0.2).Face()
circ2 = Circle((0,0),1.5).Face()

shape = circ2-circ1
mesh = Mesh(OCCGeometry(shape,dim=2).GenerateMesh(maxh=0.2))
Draw(mesh)
r = sqrt(x**2+y**2)

Gex = 2*(E+1/r)*Id(2)

gfG = GridFunction(HCurlCurl(mesh,order=6))
gfG.Set(Gex, dual=True)

Draw(Norm(Gex), mesh)
Draw(gfG.Operator("curvature"),mesh, deformation=True)